In [1]:
using Plots
using Optim
using Random
using StatsBase
using Statistics
using Distributions
using LinearAlgebra

Inicialmente, vamos elaborar funções para gerar jogadores aleatórios e simular campeonatos, além de outra para, dados os resultados, calcular a verossimilhança dos parâmetros em teste (função que deve ser maximizada).

```create_players``` recebe como parâmetros a quantidade de clubes e jogadores em cada clube e cria jogadores aleatórios.

```simulating_game``` recebe dois clubes e uma quantidade de simulações e simula os jogos entre esses dois clubes por meio de uma Poisson com média $\lambda = \frac{\lambda_a}{\lambda_d}$, onde $\lambda_a$ é o somatório das forças dos jogadores do clube atacante ($\mathcal{C}_a$) e $\lambda_d$ é o somatório das forças dos jogadores do clube defensor ($\mathcal{C}_d$).

```create_games``` recebe a quantidade de temporadas, clubes e jogadores por clube e cria jogos entre todos os clubes simulando essas temporadas.

```likelihood``` recebe os resultados e um conjunto de parâmetros e retorna a log verossimilhança negativa desses parâmetros com o resultado.

```gradient``` gradiente da função likelihood.

As duas últimas funções podem ser melhores explicadas matematicamente, dessa forma, como explicado anteriormente, cada placar é composto por duas variáveis aleatórias vindas de uma Poisson com média igual a razão das somas das forças dos dois clubes, assim seja $k$ a quantidade de gols de um clube contra outro, $\lambda_a$ a soma das forças do clube que marcou os gols e $\lambda_d$ do clube que sofreu. Dessa forma, a verossimilhança desse pedaço de placar é dada por $P(X = k | proficiências) = \frac{\lambda^k\exp{-\lambda}}{k!}$, onde $\lambda = \frac{\lambda_a}{\lambda_d}$. Note que a verossimilhança total é o produto de todos os "pedaços" de placar, mas, como cada fator desses é um valor em $[0, 1]$, isso se torna muito pequeno, podendo ocorrer underflow error, por isso opta-se a usar o log da verossimilhança (como o log é uma função estritamente crescente, a comparação entre os resultados é mantida). Por fim, os fatores se tornam parcelas, agora todas negativas, então multiplicamos por $-1$, transformando essa métrica num valor positivo e, mais importante, mudando o problema para um problema de minimização.

Agora, como citado no parágrafo anterior, cada parcela da verossimilhança é dada pela expressão $-\log{\frac{\lambda^k\exp{-\lambda}}{k!}}$, dessa forma, para cada jogador $j$, temos que a derivada dessa parcela, em relação ao jogador $j$, será:
  - $0$, se $j \notin \mathcal{C}_a \land j \notin \mathcal{C}_d$;
  - $\dfrac{\lambda_d - \lambda_a}{\lambda_d^2} + k \left(\dfrac{1}{\lambda_d} - \dfrac{1}{\lambda_a}\right)$, se $j \in \mathcal{C}_a \land j \in \mathcal{C}_d$;
  - $\dfrac{1}{\lambda_d} - \dfrac{k}{\lambda_a}$, se $j \in \mathcal{C}_a \land j \notin \mathcal{C}_d$ e;
  - $-\dfrac{\lambda_a}{\lambda_d^2} + \dfrac{k}{\lambda_d}$, se $j \notin \mathcal{C}_a \land j \in \mathcal{C}_d$.
  
Assim, passando por cada jogo e fazendo as derivadas parciais em relação a cada jogador, chegamos ao gradiente da função de verossimilhança.

In [2]:
function creat_players(clubs, players_per_club, mean = 0, var = 1, lb = 0, ub = 100)
    all_players = rand(Truncated(Normal(mean, var), lb, ub), clubs * players_per_club)
    all_players = all_players / (sum(all_players) / clubs)
    players = Dict{Int64, Float64}(zip(1:clubs * players_per_club, all_players))
    return players
end

function simulating_game(club1, club2, sims = 1000000)
    λ₁ = sum(club1)
    λ₂ = sum(club2)
    X₁ = Poisson(λ₁ / λ₂)
    X₂ = Poisson(λ₂ / λ₁)
    Y₁ = rand(X₁, sims)
    Y₂ = rand(X₂, sims)
    wins₁ = sum(Y₁ .> Y₂)
    draws = sum(Y₁ .== Y₂)
    wins₂ = sum(Y₁ .< Y₂)
    return wins₁, draws, wins₂
end

function create_games(seasons, clubs, ppc)
    results = [[[0 for i in 1:4] for j in 1:clubs * (clubs - 1)] for s in 1:seasons]
    players = creat_players(clubs, ppc)
    squads = []
    for i in 1:seasons
        line = 1
        append!(squads, [reshape(shuffle(collect(1:length(players))), (clubs, ppc))])
        actual_clubs = convert(Matrix{Float64}, (deepcopy(last(squads))))
        for i in 1:clubs
            for j in 1:ppc
                actual_clubs[i, j] = players[actual_clubs[i, j]]
            end
        end

        for j in 1:clubs
            for k in 1:clubs
                if j != k
                    Xⱼ = Poisson(sum(actual_clubs[j, :]) / sum(actual_clubs[k, :]))
                    Xₖ = Poisson(sum(actual_clubs[k, :]) / sum(actual_clubs[j, :]))
                    results[i][line][1] = j
                    results[i][line][2] = rand(Xⱼ)
                    results[i][line][3] = rand(Xₖ)
                    results[i][line][4] = k
                    line += 1
                end
            end
        end
    end
    return results, squads, players
end

# results be an array like [1, 1, 0, 3]
# [home club, score home, score away, away club]
function likelihood(players, results, squads)
    if typeof(players) != Dict{Int64, Float64}
        players = Dict{Int64, Float64}(zip(1:length(players), players))
    end
    
    loglikelihood = 0
    for i in 1:length(results)
        actual_clubs = convert(Matrix{Float64}, (deepcopy(squads[i])))
        for j in 1:size(actual_clubs)[1]
            for k in 1:size(actual_clubs)[2]
                actual_clubs[j, k] = players[actual_clubs[j, k]]
            end
        end
        
        for j in 1:length(results[i])
            clubₕ, scoreₕ, scoreₐ, clubₐ = results[i][j]
            loglikelihood -= logpdf(Poisson(sum(actual_clubs[clubₕ, :]) / sum(actual_clubs[clubₐ, :])),
                                    scoreₕ)
            loglikelihood -= logpdf(Poisson(sum(actual_clubs[clubₐ, :]) / sum(actual_clubs[clubₕ, :])),
                                    scoreₐ)
        end
    end
    
    return loglikelihood
end

function gradient(players, results, squads, ∇likelihood)
    if typeof(players) != Dict{Int64, Float64}
        players = Dict{Int64, Float64}(zip(1:length(players), players))
    end
    
    # ∇likelihood = zeros(length(players))
    for i in 1:length(results)
        for j in 1:length(results[i])
            clubₕ, scoreₕ, scoreₐ, clubₐ = results[i][j]
            λₕ, λₐ = 0, 0
            for k in 1:size(squads[i], 2)
                λₕ += players[squads[i][clubₕ, k]]
                λₐ += players[squads[i][clubₐ, k]]
            end
            
            for p in 1:length(players)
                if p in squads[i][clubₕ, :] && p in squads[i][clubₐ, :]
                    ∇likelihood[p] -= (λₕ - λₐ) / λₐ ^ 2 + scoreₕ * (λₐ - λₕ) / (λₕ * λₐ)
                    ∇likelihood[p] -= (λₐ - λₕ) / λₕ ^ 2 + scoreₐ * (λₕ - λₐ) / (λₐ * λₕ)
                elseif p in squads[i][clubₕ, :]
                    ∇likelihood[p] -= scoreₕ / λₕ - 1 / λₐ
                    ∇likelihood[p] -= λₐ / λₕ ^ 2 - scoreₐ / λₕ
                elseif p in squads[i][clubₐ, :]
                    ∇likelihood[p] -= scoreₐ / λₐ - 1 / λₕ
                    ∇likelihood[p] -= λₕ / λₐ ^ 2 - scoreₕ / λₐ
                end
            end
        end
    end
    
    return ∇likelihood
end

gradient (generic function with 1 method)

In [3]:
seasons = 10
n_clubs = 20
ppc = 11

# compilando
results, squads, players = @time create_games(seasons, n_clubs, ppc)
clubs = convert(Matrix{Float64}, (deepcopy(last(squads))))
for i in 1:n_clubs
    for j in 1:ppc
        clubs[i, j] = players[clubs[i, j]]
    end
end

@time likelihood(players, results, squads)
@time gradient(players, results, squads, zeros(length(players)))

# reexecutando
results, squads, players = @time create_games(seasons, n_clubs, ppc)
clubs = convert(Matrix{Float64}, (deepcopy(last(squads))))
for i in 1:n_clubs
    for j in 1:ppc
        clubs[i, j] = players[clubs[i, j]]
    end
end

@time likelihood(players, results, squads)
@time gradient(players, results, squads, zeros(length(players)))
println()

  0.791913 seconds (1.74 M allocations: 102.458 MiB, 2.75% gc time, 96.40% compilation time)
  0.403245 seconds (2.26 M allocations: 130.093 MiB, 10.65% gc time, 97.68% compilation time)
  1.385528 seconds (5.29 M allocations: 469.942 MiB, 6.68% gc time, 11.33% compilation time)
  0.012859 seconds (69.11 k allocations: 3.847 MiB)
  0.010405 seconds (83.26 k allocations: 3.674 MiB)
  1.208842 seconds (5.15 M allocations: 461.898 MiB, 6.63% gc time)



In [4]:
values(players)

ValueIterator for a Dict{Int64, Float64} with 220 entries. Values:
  0.10072240471694673
  0.1102340901597642
  0.07216809045036178
  0.020789349407863002
  0.058829945669204436
  0.011495914384096292
  0.1315618179953544
  0.07515827820130364
  0.014117644760884255
  0.17881652592673186
  0.07118472910996107
  0.05633387481400432
  0.10475685292540254
  0.13165294828720653
  0.12454076688923699
  0.19231654162173153
  0.11369444000296827
  0.09839665791743626
  0.03106578415120186
  0.37285057048800163
  0.1498339920768007
  0.1073773761838173
  0.09660330384068419
  0.09159127732532861
  0.008058560016609227
  ⋮

In [5]:
a = zeros(220)
opt_a = []
opt_value = Inf
for i in 1:220
    a = rand(220)
#     for j in 1:220
#         a[j] = 1 / rand(1:10)
#         println(a)
#     end
    value = likelihood(a, results, squads)
    if opt_value > value
        opt_a = a
        opt_value = value
    end
end

opt_value, likelihood(players, results, squads)

(10560.228268364872, 10005.691447429792)

# Teste para Otimização

In [6]:
f(x) = likelihood(x, results, squads)
g(∇likelihood, x) = gradient(x, results, squads, ∇likelihood)
lower = zeros(220)
upper = 20 * ones(220)
x_inicial = rand(220)
od = OnceDifferentiable(f, g, x_inicial)
@time res1  = optimize(od,
                       lower,
                       upper,
                       x_inicial,
                       Fminbox(GradientDescent()),
                       Optim.Options(iterations = 1000))
@time res2  = optimize(od,
                       lower,
                       upper,
                       x_inicial,
                       Fminbox(NelderMead()),
                       Optim.Options(iterations = 1000))
@time res3 = optimize(x -> likelihood(x, results, squads),
                      lower,
                      upper,
                      x_inicial,
                      Fminbox(NelderMead()),
                      Optim.Options(iterations = 1000))

println()

  5.482042 seconds (21.72 M allocations: 1.471 GiB, 8.51% gc time, 57.82% compilation time)
  5.969408 seconds (38.98 M allocations: 1.708 GiB, 3.93% gc time, 22.05% compilation time)
2735.770718 seconds (18.61 G allocations: 803.971 GiB, 4.13% gc time, 0.02% compilation time)



In [7]:
Optim.converged(res1), Optim.minimizer(res1), Optim.minimum(res1), likelihood(players, results, squads)

(true, [0.8064053988114606, 0.7083505250259312, 0.38896408674999017, 0.465292671883069, 0.21192459102565175, 0.05524560321118832, 0.03523755170947118, 0.6273807703084375, 0.1274308824242314, 0.8584145403039296  …  0.9076812210334986, 0.3917834086618406, 0.22962074812279298, 0.5259030236350959, 0.8970130429558458, 0.608138140231612, 0.25857237624124796, 0.6419585750004564, 0.04151790216435258, 0.8405438188487186], 10739.503593434052, 10005.691447429792)

In [8]:
max_lik_players = Dict{Int64, Float64}(zip(1:length(Optim.minimizer(res1)), Optim.minimizer(res1)))

original_players = zeros(220)
optimized_players = zeros(220)

for i in 1:220
    original_players[i] = players[i]
    optimized_players[i] = max_lik_players[i]
end

original_players /= original_players[1]
optimized_players /= optimized_players[1]
grad = gradient(Optim.minimizer(res1), results, squads, zeros(length(players)))
println("Correlação de Pearson: ", cor(original_players, optimized_players))
println("Correlação de Spearman: ", corspearman(original_players, optimized_players))
println("Correlação de Kendall: ", corkendall(original_players, optimized_players))
println("||Gradiente||: ", norm(grad))

Correlação de Pearson: 0.006183945148015716
Correlação de Spearman: -0.003596242597048399
Correlação de Kendall: -0.0021585720215857203
||Gradiente||: 329.68311552961467


In [9]:
Optim.converged(res2), Optim.minimizer(res2), Optim.minimum(res2), likelihood(players, results, squads)

(true, [0.8064053988114606, 0.7083505250259312, 0.38896408674999017, 0.465292671883069, 0.21192459102565175, 0.05524560321118832, 0.03523755170947118, 0.6273807703084375, 0.1274308824242314, 0.8584145403039296  …  0.9076812210334986, 0.3917834086618406, 0.22962074812279298, 0.5259030236350959, 0.8970130429558458, 0.608138140231612, 0.25857237624124796, 0.6419585750004564, 0.04151790216435258, 0.8405438188487186], 10719.28862408449, 10005.691447429792)

In [10]:
max_lik_players = Dict{Int64, Float64}(zip(1:length(Optim.minimizer(res2)), Optim.minimizer(res2)))

original_players = zeros(220)
optimized_players = zeros(220)

for i in 1:220
    original_players[i] = players[i]
    optimized_players[i] = max_lik_players[i]
end

original_players /= original_players[1]
optimized_players /= optimized_players[1]
grad = gradient(Optim.minimizer(res2), results, squads, zeros(length(players)))
println("Correlação de Pearson: ", cor(original_players, optimized_players))
println("Correlação de Spearman: ", corspearman(original_players, optimized_players))
println("Correlação de Kendall: ", corkendall(original_players, optimized_players))
println("||Gradiente||: ", norm(grad))

Correlação de Pearson: 0.01722695267248364
Correlação de Spearman: 0.003408034350822425
Correlação de Kendall: 0.0026567040265670404
||Gradiente||: 320.7019309829996


In [11]:
Optim.converged(res3), Optim.minimizer(res3), Optim.minimum(res3), likelihood(players, results, squads)

(true, [0.10937194795810211, 0.1151962451481414, 0.9619587164046001, 0.17523075403670446, 1.5251654103600665, 1.6048909139233274, 0.25041128991141165, 1.7855244433862993, 0.8278193898836703, 1.2792880362530639  …  0.9583233504478451, 0.7297231595761781, 0.18453431116544816, 0.28092527527031524, 0.038379377223698656, 1.2791509792165767, 0.4913336107669751, 0.022856400494661686, 1.0609738818211167, 1.028822744697991], 9918.517213704947, 10005.691447429792)

In [12]:
max_lik_players = Dict{Int64, Float64}(zip(1:length(Optim.minimizer(res3)), Optim.minimizer(res3)))

original_players = zeros(220)
optimized_players = zeros(220)

for i in 1:220
    original_players[i] = players[i]
    optimized_players[i] = max_lik_players[i]
end

original_players /= original_players[1]
optimized_players /= optimized_players[1]
grad = gradient(Optim.minimizer(res3), results, squads, zeros(length(players)))
println("Correlação de Pearson: ", cor(original_players, optimized_players))
println("Correlação de Spearman: ", corspearman(original_players, optimized_players))
println("Correlação de Kendall: ", corkendall(original_players, optimized_players))
println("||Gradiente||: ", norm(grad))

Correlação de Pearson: 0.7390958298104948
Correlação de Spearman: 0.6963693840406169
Correlação de Kendall: 0.5001245330012454
||Gradiente||: 8.332437545008586


In [13]:
@time res4  = optimize(od,
                       lower,
                       upper,
                       Optim.minimizer(res3),
                       Fminbox(GradientDescent()),
                       Optim.Options(iterations = 1000))

res4

  3.127029 seconds (10.54 M allocations: 934.922 MiB, 4.85% gc time)


 * Status: success

 * Candidate solution
    Final objective value:     9.918517e+03

 * Found with
    Algorithm:     Fminbox with Gradient Descent

 * Convergence measures
    |x - x'|               = 0.00e+00 ≤ 0.0e+00
    |x - x'|/|x'|          = 0.00e+00 ≤ 0.0e+00
    |f(x) - f(x')|         = NaN ≰ 0.0e+00
    |f(x) - f(x')|/|f(x')| = NaN ≰ 0.0e+00
    |g(x)|                 = NaN ≰ 1.0e-08

 * Work counters
    Seconds run:   3  (vs limit Inf)
    Iterations:    1
    f(x) calls:    2
    ∇f(x) calls:   2


In [14]:
Optim.minimizer(res3) == Optim.minimizer(res4)

true